In [8]:
%pip install langchain-core langchain-openai requests pillow


[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
# Setup
from dotenv import load_dotenv
import os
import base64
import requests
load_dotenv()
open_ai_key = os.getenv('OPENAI_API_KEY')
# image_path = "./storage/info_confirm.png"
image_path = "./storage/sign_up.png"
filename = os.path.basename(image_path)

In [32]:
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

In [33]:
# Getting the base64 string
base64_image = encode_image_to_base64(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {open_ai_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "You are an expert software developer. You have been tasked with creating a HTML and CSS version of this user interface wireframe. You have been given the following image as a reference. Only include the full HTML and CSS. Do not truncate it. Do not include a response prompt or markdown formatting."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 2000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [34]:

# Parse the response and give user feedback
response_json = response.json();
print("Response", response_json)
tokens = response_json['usage']['prompt_tokens']
print("Tokens used", tokens)
content = response_json['choices'][0]['message']['content']

Response {'id': 'chatcmpl-9lZJG4uDXvP3ZPC0PLfjVVQTypR4V', 'object': 'chat.completion', 'created': 1721124174, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Sign Up Confirmation</title>\n    <style>\n        * {\n            box-sizing: border-box;\n            margin: 0;\n            padding: 0;\n        }\n        \n        body, html {\n            height: 100%;\n            display: flex;\n            align-items: center;\n            justify-content: center;\n            font-family: Arial, sans-serif;\n            background-color: #f0f0f0;\n        }\n        \n        .container {\n            background-color: #d3d3d3;\n            width: 300px;\n            padding: 20px;\n            border-radius: 10px;\n            text-align: center;\n            position: relative

In [35]:
def strip_after_html_tag(response_text):
    # Find the index of the first occurrence of ```html
    start_index = response_text.find("```html")
    
    if start_index != -1:
        # Find the index of the next occurrence of ``` (to close the HTML code block)
        end_index = response_text.find("```", start_index + len("```html"))
        
        if end_index != -1:
            # Include the closing tag
            end_index += len("```")
            # Return the substring up to the closing tag of the HTML block
            return response_text[:end_index]
    
    # If the tags are not found, return the original text
    return response_text

# Write the output to storage
html_content = strip_after_html_tag(content)
# print(html_content)
file_path = f"./storage/output/{filename}.html"

f = open(file_path, "w")
f.write(html_content)
f.close()